### import some libraries

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from notebookutils import mssparkutils
import pandas as pd

StatementMeta(, 05610a8f-a02a-4709-a8d4-169f9ac518b6, 3, Finished, Available)

In [6]:
mssparkutils.fs.mount("abfss://fb9f6c83-5ff0-46a3-9b49-0a37dbafa509@onelake.dfs.fabric.microsoft.com/fac3800b-7a90-47b1-ab86-fa4ccea42f00/Files","/Files")

mssparkutils.fs.getMountPath("/Files")

check_files=mssparkutils.fs.ls(f"file://{mssparkutils.fs.getMountPath('/Files')}/Current/")

check_files


StatementMeta(, 05610a8f-a02a-4709-a8d4-169f9ac518b6, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9170d585-a985-4c3e-bff1-94589883217f)

[FileInfo(path=file:/synfs/notebook/05610a8f-a02a-4709-a8d4-169f9ac518b6/Files/Current/Sales_02012023.xlsx, name=Sales_02012023.xlsx, size=498608),
 FileInfo(path=file:/synfs/notebook/05610a8f-a02a-4709-a8d4-169f9ac518b6/Files/Current/Sales_04012023.xlsx, name=Sales_04012023.xlsx, size=535106),
 FileInfo(path=file:/synfs/notebook/05610a8f-a02a-4709-a8d4-169f9ac518b6/Files/Current/completedacct.csv, name=completedacct.csv, size=233455)]

In [7]:
#Get only the names of files that start with 'Sales' in the 'Current
file_names = [file_info.name for file_info in check_files if file_info.name.startswith("Sales")]
file_names

StatementMeta(, 05610a8f-a02a-4709-a8d4-169f9ac518b6, 9, Finished, Available)

['Sales_02012023.xlsx', 'Sales_04012023.xlsx']

In [4]:
folder_path = "abfss://fb9f6c83-5ff0-46a3-9b49-0a37dbafa509@onelake.dfs.fabric.microsoft.com/fac3800b-7a90-47b1-ab86-fa4ccea42f00/Files/Current/"

StatementMeta(, 3097e8d6-0cb9-4085-ae9a-32096df19d59, 6, Finished, Available)

In [5]:
dfs = []
for file_name in file_names:
    file_path = folder_path + file_name
    df = pd.read_excel(file_path, sheet_name="Sales")
    dfs.append(df)

df_sales = spark.createDataFrame(pd.concat(dfs))
df_sales.count()

StatementMeta(, 3097e8d6-0cb9-4085-ae9a-32096df19d59, 7, Finished, Available)

4365

In [7]:
dfr = []
for file_name in file_names:
    file_path = folder_path + file_name
    df = pd.read_excel(file_path, sheet_name="Returns")
    dfr.append(df)

df_returns = spark.createDataFrame(pd.concat(dfr))
df_returns.count()

StatementMeta(, 3097e8d6-0cb9-4085-ae9a-32096df19d59, 9, Finished, Available)

50

In [8]:
df_final=df_sales.join(df_returns,df_sales.Order_ID==df_returns.Order_ID,how="left").drop(df_returns.Order_ID,df_returns.Customer_Name,df_returns.Sales_Amount)
## combine df_sales with df_returns. Delete duplicated column in df_returns. 

StatementMeta(, 3097e8d6-0cb9-4085-ae9a-32096df19d59, 10, Finished, Available)

In [9]:
df_mod=df_final.withColumns({"Order_Year": year("Order_Date"), \
                "Order_Month": month("Order_Date"), \
                "Created_TS": current_timestamp(),\
                "Modified_TS": current_timestamp(),\
                })

StatementMeta(, 3097e8d6-0cb9-4085-ae9a-32096df19d59, 11, Finished, Available)

In [10]:
df_mod.createOrReplaceTempView("ViewSale")

StatementMeta(, 3097e8d6-0cb9-4085-ae9a-32096df19d59, 12, Finished, Available)

### Create Table Bronze_Sales

In [18]:
%%sql
create table if not exists lh_ecormece.Bronze_Sales
(
    Order_ID	string,
    Order_Date	Date,
    Shipping_Date	date,
    Aging	int,
    Ship_Mode	string,
    Product_Category	string,
    Product	 string,
    Sales 	float,
    Quantity	float,
    Discount	 float,
    Profit 	 float,
    Shipping_Cost 	float,
    Order_Priority	string,
    Customer_ID	string,
    Customer_Name	string,
    Segment	string,
    City	string,
    State	string,
    Country	string,
    Region string,
	Return string,
    Order_Year int,
    Order_Month int,
    Created_TS  TIMESTAMP,
    Modified_TS  TIMESTAMP
)
using DELTA
PARTITIONED by(Order_Year,Order_Month)



StatementMeta(, 22595a90-c6fb-40d8-a9b2-dcb66147a3ee, 20, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

### Load data from ViewSale to Bronze_Sales

In [20]:
%%sql
Merge into  lh_ecormece.bronze_sales as BS
using ViewSale as VS
on  BS.Order_Year=VS.Order_Year and BS.Order_Month=VS.Order_Month and BS.Order_ID=VS.Order_ID
when matched then 
update SET
BS.Order_Date	=	VS.Order_Date	,
BS.Shipping_Date	=	VS.Shipping_Date	,
BS.Aging	=	VS.Aging	,
BS.Ship_Mode	=	VS.Ship_Mode	,
BS.Product_Category	=	VS.Product_Category	,
BS.Product	=	VS.Product	,
BS.Sales 	=	VS.Sales 	,
BS.Quantity	=	VS.Quantity	,
BS.Discount	=	VS.Discount	,
BS.Profit 	=	VS.Profit 	,
BS.Shipping_Cost 	=	VS.Shipping_Cost 	,
BS.Order_Priority	=	VS.Order_Priority	,
BS.Customer_ID	=	VS.Customer_ID	,
BS.Customer_Name	=	VS.Customer_Name	,
BS.Segment	=	VS.Segment	,
BS.City	=	VS.City	,
BS.State	=	VS.State	,
BS.Country	=	VS.Country	,
BS.Region  	=	VS.Region  	,
BS.Return  	=	VS.Return  	,
BS.Modified_TS	=	VS.Modified_TS	

when not matched then 
INSERT
(
BS.Order_ID,    
BS.Order_Date	,
BS.Shipping_Date	,
BS.Aging	,
BS.Ship_Mode	,
BS.Product_Category	,
BS.Product	,
BS.Sales 	,
BS.Quantity	,
BS.Discount	,
BS.Profit 	,
BS.Shipping_Cost 	,
BS.Order_Priority	,
BS.Customer_ID	,
BS.Customer_Name	,
BS.Segment	,
BS.City	,
BS.State	,
BS.Country	,
BS.Region  	,
BS.Return,
BS.Order_Year  	,
BS.Order_Month  	,
BS.Created_TS   	,
BS.Modified_TS	
)
values
(
VS.Order_ID,
VS.Order_Date	,
VS.Shipping_Date	,
VS.Aging	,
VS.Ship_Mode	,
VS.Product_Category	,
VS.Product	,
VS.Sales 	,
VS.Quantity	,
VS.Discount	,
VS.Profit 	,
VS.Shipping_Cost 	,
VS.Order_Priority	,
VS.Customer_ID	,
VS.Customer_Name	,
VS.Segment	,
VS.City	,
VS.State	,
VS.Country	,
VS.Region  	,
VS.Return,
VS.Order_Year  	,
VS.Order_Month  	,
VS.Created_TS   	,
VS.Modified_TS	

)


StatementMeta(, 22595a90-c6fb-40d8-a9b2-dcb66147a3ee, 22, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>